# POS tagging using HMM

In this assignment, part-of-speech tagging has been performed using Hidden Markov model. The model is implemented using own approach from scratch and using NLTK package preexisting model.

1.   Accuracy using own approach - 87.87%
2.   Accuracy with NLTK HMM model - 92.05%

**Please find the [flowchart](https://drive.google.com/file/d/1j1gpVV2PryZaeShIY3-O9ORUo6GrKPls/view?usp=sharing) of this approach** 

**Importing all the required packages and libraries**

In [1]:
import numpy as np
import nltk
from sklearn.model_selection import train_test_split
import pandas as pd
from nltk.corpus import brown
import random
import statistics
import warnings
warnings.filterwarnings("ignore")
nltk.download('brown')
nltk.download('universal_tagset')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


True

# Obtaining the dataset in required formats

*   'brown_news_tagged' contains (word,tag) pairs of news category data with universal tagset
*   'words' contains only the words of brown_news_tagged
*   'tags' and 'tags_list' contains only the tags of brown_news_tagged (with repetation)
*   'unique_tags' contains unique tags of the entire dataset
*   'vocab' contains unique words of the dataset







In [2]:
brown_news_tagged = brown.tagged_words(categories='news', tagset='universal')
words = brown.words(categories='news')
tags = []
for i in range(len(brown_news_tagged)):
  tags.append(brown_news_tagged[i][1])

In [3]:
print(brown_news_tagged[:5])
print(words[:5])
print(tags[:5])

[('The', 'DET'), ('Fulton', 'NOUN'), ('County', 'NOUN'), ('Grand', 'ADJ'), ('Jury', 'NOUN')]
['The', 'Fulton', 'County', 'Grand', 'Jury']
['DET', 'NOUN', 'NOUN', 'ADJ', 'NOUN']


In [4]:
print(len(words))
print(len(tags))

100554
100554


In [7]:
unique_tags = list({tag for word,tag in train_set})
tags_list = []
for i in range(len(train_set)):
  tags_list.append(train_set[i][1])
vocab = {word for word,tag in train_set}

# Obtaining the train set and test set using train_test_spit

In [5]:
# split data into training and validation set in the ratio 80:20
train_set,test_set =train_test_split(brown_news_tagged,train_size=0.80,test_size=0.20)

In [6]:
train_set[0:5]

[('The', 'DET'), ('.', '.'), ('the', 'DET'), ('we', 'PRON'), ('that', 'DET')]

# Function for calculating Transition probability of consecutive tag pairs

In [8]:
# Calculating occurance of tag t2 after tag t1. Later to be used for transition probability

def t2_given_t1(t2, t1, tags_list):
  
  t1_count = 0
  t2_after_t1 = 0
  for i in range(len(tags_list)-1):
    # Counting the number of times tag t1 occurs
    if tags_list[i] == t1:
      t1_count += 1
      # Counting the number of times tag t2 occurs after tag t1
      if tags_list[i+1] == t2:
        t2_after_t1 += 1

  return t2_after_t1/t1_count

# Construction tags matrix using transition probability function

In [9]:
# Creating n x n transition matrix of unique_tags
 
tags_matrix = np.zeros((len(unique_tags), len(unique_tags)), dtype='float32')
i = 0
for t1 in unique_tags:
  j = 0
  for t2 in unique_tags: 
    # Matrix(i, j) represents P(jth tag after the ith tag)
    tags_matrix[i, j] = t2_given_t1(t2, t1, tags_list)
    j += 1
  i += 1

In [10]:
tags_df = pd.DataFrame(tags_matrix, columns = list(unique_tags), index=list(unique_tags))
display(tags_df)

ADP      CONJ         .  ...      VERB         X       DET
ADP   0.123169  0.028377  0.121033  ...  0.140460  0.001017  0.113609
CONJ  0.113407  0.027548  0.129017  ...  0.142792  0.000000  0.110652
.     0.119949  0.025253  0.118792  ...  0.143624  0.001052  0.112163
ADJ   0.122940  0.028143  0.110720  ...  0.147010  0.000926  0.112942
NOUN  0.123254  0.027005  0.116085  ...  0.143905  0.001018  0.113804
PRT   0.129834  0.023204  0.116022  ...  0.152486  0.000552  0.119890
NUM   0.114620  0.023392  0.125146  ...  0.140936  0.001170  0.113450
ADV   0.124436  0.030827  0.129323  ...  0.128195  0.000752  0.116165
PRON  0.121987  0.029021  0.113133  ...  0.131825  0.001968  0.108214
VERB  0.119650  0.025732  0.114365  ...  0.148501  0.000866  0.116011
X     0.040000  0.053333  0.080000  ...  0.160000  0.000000  0.160000
DET   0.125847  0.028428  0.123770  ...  0.142795  0.000656  0.112180

[12 rows x 12 columns]

# Function for calculating Emission probability of word tag pairs

In [11]:
# Computing Emission Probability

def word_given_tag(word, tag, train_set):
 
  spec_t_count = 0
  spec_w_count = 0
  for i in range(len(train_set)):
    # Calculating the total number of times the passed tag occured
    if train_set[i][1] == tag:
      spec_t_count += 1
      # Calculating the total number of times the passed word occurred as the passed tag
      if train_set[i][0] == word:
        spec_w_count += 1
  return spec_w_count/spec_t_count

# Viterbi function for obtaining most appropriate word tag pair
Viterbi function basically finds the final probability - emission * transmission for every word-tag pair and finds the maximum probability out of this pool. 

In [12]:
def viterbi(words, unique_tags, train_set):

  pred = []
  for i in range(len(words)):
    # Initializing list of probability list for a given pair
    p = [] 
    for j in range(len(unique_tags)):
      # transition probability for first word and first tag can't be calculated
      if i == 0: transition = tags_df.loc['.', unique_tags[j]]
      # transition probability calculation based on last chosen tag
      else: transition = tags_df.loc[pred[-1], unique_tags[j]]
            
      # Computing emission and final probabilities
      emission = word_given_tag(words[i], unique_tags[j], train_set)
      pred_probability = transition * emission    
      p.append(pred_probability)
        
    # Fetting the tag for the pair with maximum probability
    pred_max = unique_tags[p.index(max(p))] 
    pred.append(pred_max)
  return list(zip(words, pred))

# Selecting 50 random word tag pairs out of test set for testing
After obtaining 50 such pairs, we obtain tagless words and measure the accuracy of out predicted tags.

In [13]:
# Choose random 10 numbers for testing
random_list = [random.randint(1,len(test_set)) for x in range(50)]
test_run = []
for i in random_list:
  test_run.append(test_set[i])
# list of 10 pairs on which we test the model
test_run = [test_set[i] for i in random_list]

test_run_base = []
for i in range(len(test_run)):
  test_run_base.append(test_run[i][0])

In [14]:
pred_sub = viterbi(test_run_base, unique_tags, train_set) 
# accuracy
res_sub = []
for i, j in zip(pred_sub, test_run):
  if i == j: res_sub.append(i) 
accuracy = len(res_sub)/len(pred_sub)
print('Accuracy of own approach on 50 randomly chosen words: ',accuracy*100)

Accuracy of own approach on 50 randomly chosen words:  94.0


# Testing our approach on entire test set

**Obtaining tagless words for experimentation**

In [15]:
test_untagged = []
for i in range(len(test_set)):
  test_untagged.append(test_set[i][0])

**Obtaining tags for untagged test set in the batches of 2000**
Here, experimentation was done with different batch sizes and following results were obtained.
*   Batch size 500 - 87.49
*   Batch size 2000 - 87.87
*   Batch size 5000 - 88.03



In [27]:
size = 2000
start = 0
end = min(size,len(test_untagged) - start)
accuracy = []
for k in range(len(test_untagged)//size + 1):
  res = []
  pred_set = viterbi(test_untagged[start:end], unique_tags, train_set)
  for i, j in zip(pred_set, test_set[start:end]):
    if i == j: res.append(i) 
  accuracy.append((len(res)/len(pred_set))*100)
  start = start + size
  end = start + min(size,len(test_untagged) - start)
  print("Accuracy for batch no.", k+1, ":", round(accuracy[k],2))

Accuracy for batch no. 1 : 87.75
Accuracy for batch no. 2 : 87.25
Accuracy for batch no. 3 : 87.2
Accuracy for batch no. 4 : 87.8
Accuracy for batch no. 5 : 87.3
Accuracy for batch no. 6 : 88.1
Accuracy for batch no. 7 : 86.85
Accuracy for batch no. 8 : 88.15
Accuracy for batch no. 9 : 89.65
Accuracy for batch no. 10 : 87.35
Accuracy for batch no. 11 : 89.19


**Obtaining average accuracy of all batches**

In [28]:
print('Accuracy using own HMM trainer: ',round(statistics.mean(accuracy),2))

Accuracy using own HMM trainer:  87.87


# Performing HMM on test set using NLTK pre-existing model

In [23]:
from nltk.tag import hmm

tagger = nltk.HiddenMarkovModelTagger.train([train_set])
tagger.tag(test_untagged)
print("Accuracy using NLTK HMM trainer:", round(tagger.evaluate([test_set])*100,2))

Accuracy using NLTK HMM trainer: 92.05
